In [1]:
#------------------- Change working directory to project root -------------------#
from pathlib import Path, os

cur = Path().resolve()
while not (cur / "src").is_dir():
    if cur == cur.parent: raise RuntimeError("No 'src' dir")
    cur = cur.parent

os.chdir(cur)
print(f"[INFO] Changed working directory to project root: {cur}")

[INFO] Changed working directory to project root: /home/fernando/Documents/LoRaPriv


In [2]:
# -------------------------------------- External Libraries --------------------------------------
import numpy as np
from datetime import datetime
from typing import Callable
import matplotlib.pyplot as plt
import random
from collections import defaultdict
import time
# ----------------------------------------- Local Imports ----------------------------------------
from src.core                  import LoRaPhyParams, LoRaFrameParams
from src.mod                   import LoRaModulator
from src.demod                 import LoRaDemodulator
from src.sync                  import DechirpBasedSynchronizer
from src.core.vpn_utils        import VPNKeepAlive

import src.core.sdr_utils as sdr_utils
import src.core.snr_utils as snr_utils
import src.core.perf_metrics_utils as perf_utils


In [7]:
MOD_BACKEND = "numpy"  
DEMOD_BACKEND = "cupy" 
payload_syms_count = 1000
SIMULATIONS_BUFFER_SIZE = 2**20  # Tamaño del buffer para simulaciones

class ReceivedSyncError(Exception):
    """Excepción para errores de sincronización recibidos."""
    def __init__(self, message):
        super().__init__(message)
        self.message = message

    def __str__(self):
        return f"ReceivedSyncError: {self.message}"

def run(snr_db):

    profile_name = "sf7_500k_2spc_0fpa.json"
    profile = snr_utils.SDRProfile.load(f"lora_sim/{profile_name}")

    modulator=LoRaModulator(profile.phy_params, profile.frame_params, backend=MOD_BACKEND)

    synchronizer=DechirpBasedSynchronizer(profile.phy_params, profile.frame_params, backend=DEMOD_BACKEND, fold_mode=profile.fold_mode, max_sync_candidates=50, debug=True, compensate_cfo_sfo=True)

    demodulator=LoRaDemodulator(profile.phy_params, backend=DEMOD_BACKEND, fold_mode=profile.fold_mode)

    payload = np.random.randint(0, modulator.phy_params.chips_per_symbol, size=payload_syms_count)
    modulated_frame = modulator.modulate(payload, include_frame=True)
    reference_payload = modulator.modulate(payload, include_frame=False)

    # Añade ruido blanco gaussiano al frame modulado
    noisy_frame, _, _ = snr_utils.generate_awgn(f"{snr_db}db", modulated_frame, reference_payload)

    # Repite el frame para que tenga el tamaño de un buffer
    buffer_size = SIMULATIONS_BUFFER_SIZE


    sync_offset = np.random.randint(0, buffer_size - len(noisy_frame))
    #print(f"[INFO] Sync offset: {sync_offset} samples (or {sync_offset + modulator.phy_params.samples_per_symbol} samples)")

    if len(noisy_frame) < buffer_size:
        reps = (buffer_size // len(noisy_frame)) + 1
        noisy_frame = np.tile(noisy_frame, reps)

    # Desplaza el frame para simular un canal real y un offset aleatorio
    received_iq = np.roll(noisy_frame, sync_offset)

    #print(f"[INFO] Other possible frame starts: {np.arange(sync_offset, len(received_iq) - len(modulated_frame), modulated_frame.size)}")

    aligned_payload, traces = synchronizer.run(received_iq)
    demodulated_symbols = demodulator.demodulate(aligned_payload)
    demodulated_symbols = demodulated_symbols.get() if hasattr(demodulated_symbols, 'get') else demodulated_symbols

    if len(demodulated_symbols) != len(payload):
        raise ReceivedSyncError(f"Payload mismatch: {demodulated_symbols} != {payload}")

    # Calcula Errores
    ser_err = np.sum(demodulated_symbols != np.asarray(payload))
    ber_err = sum(bin(m ^ dm).count('1') for m, dm in zip(payload, demodulated_symbols))
    return ser_err, ber_err, traces

In [8]:
from src.sync.dechirp_based_synchronizer import CandidatesExhaustedError, NoCandidatesFoundError

from collections import Counter

# ---- Parameters ----
snr_db = -10
NUM_ITERS = 1000  # adjust as needed
no_candidates_found = 0
received_sync_errors = 0
candidates_exhausted = 0
all_traces = []

total_ser_err = 0
total_ber_err = 0
for _ in range(NUM_ITERS):
    try:
        ser_err, ber_err, traces = run(snr_db)
        # Optional: print per-run error metrics
        total_ser_err += ser_err
        total_ber_err += ber_err
    except NoCandidatesFoundError as e:
        traces = []
        no_candidates_found += 1
    except CandidatesExhaustedError as e:
        traces = getattr(e, "traces", []) or []
        candidates_exhausted += 1
    except ReceivedSyncError as e:
        traces = []
        received_sync_errors += 1

    all_traces.extend(traces or [])

# ---- Summary of most frequent errors across all iterations ----
def summarize_errors(traces):
    """
    Build minimal frequency summaries using only fields present in traces:
    - error_type
    - error_msg
    - index (candidate index)
    - status ("success" | "error")
    """
    total = len(traces)
    failed = [t for t in traces if getattr(t, "status", "error") != "success"]

    by_type = Counter(getattr(t, "error_type", "-") or "-" for t in failed)
    by_type_msg = Counter(
        ((getattr(t, "error_type", "-") or "-"), (getattr(t, "error_msg", "-") or "-"))
        for t in failed
    )
    by_index = Counter(getattr(t, "index", -1) for t in failed)

    print("\n=== Error summary across runs ===")
    print(f"Total candidate attempts : {total}")
    print(f"Failed candidates       : {len(failed)} ({(100.0*len(failed)/total if total else 0.0):.1f}%)")

    if by_type:
        top_type, top_cnt = by_type.most_common(1)[0]
        print(f"\nMost frequent error type: {top_type} ({top_cnt})")

        # Show the most frequent message for that type
        msg_counts = Counter({msg: cnt for (etype, msg), cnt in by_type_msg.items() if etype == top_type})
        if msg_counts:
            top_msg, top_msg_cnt = msg_counts.most_common(1)[0]
            print(f"Top message for '{top_type}': {top_msg} ({top_msg_cnt})")

    if by_index:
        idx, cnt = by_index.most_common(1)[0]
        print(f"\nMost failing candidate index: {idx} ({cnt})")



print(f"\nSent {NUM_ITERS} frames with SNR {snr_db} dB")
print(f"Successful frames: {NUM_ITERS - no_candidates_found - received_sync_errors - candidates_exhausted}")
print(f"FER: {(received_sync_errors + candidates_exhausted + no_candidates_found) / NUM_ITERS if NUM_ITERS > 0 else 0:.2%}")
print(f"SER: {total_ser_err/(NUM_ITERS*1000)}, BER: {total_ber_err/(NUM_ITERS*7000)}")
print(f"No candidates found: {no_candidates_found} times")
print(f"Payload didn't match: {received_sync_errors} times")
print(f"Candidates exhausted: {candidates_exhausted} times")

print(f"\n Why did candidates exhaust? See error summary below:\n")
# Run the summary
summarize_errors(all_traces)



Sent 1000 frames with SNR -10 dB
Successful frames: 817
FER: 18.30%
SER: 0.031387, BER: 0.015819714285714284
No candidates found: 23 times
Payload didn't match: 43 times
Candidates exhausted: 117 times

 Why did candidates exhaust? See error summary below:


=== Error summary across runs ===
Total candidate attempts : 1319
Failed candidates       : 502 (38.1%)

Most frequent error type: SFDError (368)
Top message for 'SFDError': Failed to locate the downchirp pair in SFD. (368)

Most failing candidate index: 0 (117)


In [6]:
from src.sync.dechirp_based_synchronizer import CandidatesExhaustedError, NoCandidatesFoundError

from collections import Counter

# ---- Parameters ----
snr_db = -10
NUM_ITERS = 1000  # adjust as needed
no_candidates_found = 0
received_sync_errors = 0
candidates_exhausted = 0
all_traces = []

total_ser_err = 0
total_ber_err = 0
for _ in range(NUM_ITERS):
    try:
        ser_err, ber_err, traces = run(snr_db)
        # Optional: print per-run error metrics
        total_ser_err += ser_err
        total_ber_err += ber_err
    except NoCandidatesFoundError as e:
        traces = []
        no_candidates_found += 1
    except CandidatesExhaustedError as e:
        traces = getattr(e, "traces", []) or []
        candidates_exhausted += 1
    except ReceivedSyncError as e:
        traces = []
        received_sync_errors += 1

    all_traces.extend(traces or [])

# ---- Summary of most frequent errors across all iterations ----
def summarize_errors(traces):
    """
    Build minimal frequency summaries using only fields present in traces:
    - error_type
    - error_msg
    - index (candidate index)
    - status ("success" | "error")
    """
    total = len(traces)
    failed = [t for t in traces if getattr(t, "status", "error") != "success"]

    by_type = Counter(getattr(t, "error_type", "-") or "-" for t in failed)
    by_type_msg = Counter(
        ((getattr(t, "error_type", "-") or "-"), (getattr(t, "error_msg", "-") or "-"))
        for t in failed
    )
    by_index = Counter(getattr(t, "index", -1) for t in failed)

    print("\n=== Error summary across runs ===")
    print(f"Total candidate attempts : {total}")
    print(f"Failed candidates       : {len(failed)} ({(100.0*len(failed)/total if total else 0.0):.1f}%)")

    if by_type:
        top_type, top_cnt = by_type.most_common(1)[0]
        print(f"\nMost frequent error type: {top_type} ({top_cnt})")

        # Show the most frequent message for that type
        msg_counts = Counter({msg: cnt for (etype, msg), cnt in by_type_msg.items() if etype == top_type})
        if msg_counts:
            top_msg, top_msg_cnt = msg_counts.most_common(1)[0]
            print(f"Top message for '{top_type}': {top_msg} ({top_msg_cnt})")

    if by_index:
        idx, cnt = by_index.most_common(1)[0]
        print(f"\nMost failing candidate index: {idx} ({cnt})")



print(f"\nSent {NUM_ITERS} frames with SNR {snr_db} dB")
print(f"Successful frames: {NUM_ITERS - no_candidates_found - received_sync_errors - candidates_exhausted}")
print(f"FER: {(received_sync_errors + candidates_exhausted + no_candidates_found) / NUM_ITERS if NUM_ITERS > 0 else 0:.2%}")
print(f"SER: {total_ser_err/(NUM_ITERS*1000)}, BER: {(total_ber_err)/(NUM_ITERS*7000)}")
print(f"No candidates found: {no_candidates_found} times")
print(f"Payload didn't match: {received_sync_errors} times")
print(f"Candidates exhausted: {candidates_exhausted} times")

print(f"\n Why did candidates exhaust? See error summary below:\n")
# Run the summary
summarize_errors(all_traces)



Sent 1000 frames with SNR -10 dB
Successful frames: 913
FER: 8.70%
SER: 0.084742, BER: 0.030260285714285715
No candidates found: 12 times
Payload didn't match: 55 times
Candidates exhausted: 20 times

 Why did candidates exhaust? See error summary below:


=== Error summary across runs ===
Total candidate attempts : 986
Failed candidates       : 73 (7.4%)

Most frequent error type: IncompletePayloadError (40)
Top message for 'IncompletePayloadError': IQ buffer too short to extract as many payload symbols as the header indicates. (40)

Most failing candidate index: 0 (24)
